In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [2]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader

data_loader = HypergraphLoader(config)
data = data_loader.load()

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072
Final num_hyperedges 1392
Final num_nodes 2708
Final num_class 7


### Loss

In [3]:
loss = hydra.utils.instantiate(config.model.loss)

### Backbone

In [4]:
hydra.utils.instantiate(config.model.backbone)

AllSetTransformer(
  (layers): ModuleList(
    (0): AllSetTransformerLayer(
      (vertex2edge): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=128, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (edge2vertex): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=128, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)

### ReadOut

In [4]:
readout = hydra.utils.instantiate(config.model.readout)

In [5]:
readout.parameters()

<generator object Module.parameters at 0x7fe978c5a6c0>

### Evaluator

In [2]:
evaluator = hydra.utils.instantiate(config.evaluator)

In [4]:
import torch

d = {
    "labels": torch.tensor([0, 1, 2, 1, 2, 2]),
    "logits": torch.tensor(
        [[1, 11, 11], [5, 11, 3], [2, 3, 4], [5, 16, 7], [8, 9, 10], [11, 12, 13]]
    ),
}


evaluator.eval(d)

/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'acc': 0.8333333333333334,
 'pre_micro': 0.8333333333333334,
 'pre_macro': 0.5555555555555555,
 'rec_micro': 0.8333333333333334,
 'rec_macro': 0.6666666666666666,
 'f1_micro': 0.8333333333333334,
 'f1_macro': 0.6}

In [8]:
d["logits"].argmax(dim=1)

tensor([0, 1, 2, 1, 2, 2])

### Dataloader

In [3]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule

data_loader = HypergraphLoader(config)
data = data_loader.load()
dataloader = FullBatchDataModule(data=data)

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


In [5]:
batch = next(iter(dataloader.train_dataloader()))

In [12]:
batch.x[batch.train_mask]

torch.Size([1354, 1433])

In [2]:
a = hydra.utils.instantiate(config.model)

/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout_workaround' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout_workaround'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout'])`.


In [9]:
a.hparams.backbone is a.backbone

True

In [13]:
id(a.hparams.backbone) == id(a.backbone)

True

In [22]:
b = a.backbone.__class__

In [31]:
b in []topomodelx.nn.hypergraph.unigcnii.UniGCNII

True

In [29]:
import topomodelx